# **Leetcode Problem Recommendation System**

## **Data Pre-Processing**

In [1]:
import json
from collections import defaultdict
import pandas as pd
import numpy as np

data = json.loads(open("data.json", "r").read())
new_data = defaultdict(list)
for key in data.keys():
    for attribute in data[key].keys():
        new_data[attribute].append(data[key][attribute])

with open("updated_data.json", 'w+') as f:
    f.write(json.dumps(new_data))

df = pd.DataFrame(new_data)
df.head()

,accuracy,title,titleSlug,topics,difficulty,questionId,questionFrontendId,question,link,likes,dislikes,likability
0,54.199137,Two Sum,two-sum,"[array, hash-table]",easy,1,1,<p>Given an array of integers <code>nums</code...,https://leetcode.com/problems/two-sum,58795,2097,96.556198
1,44.566495,Add Two Numbers,add-two-numbers,"[linked-list, math, recursion]",medium,2,2,<p>You are given two <strong>non-empty</strong...,https://leetcode.com/problems/add-two-numbers,32082,6445,83.271472
2,35.730692,Longest Substring Without Repeating Characters,longest-substring-without-repeating-characters,"[hash-table, string, sliding-window]",medium,3,3,"<p>Given a string <code>s</code>, find the len...",https://leetcode.com/problems/longest-substrin...,40582,1956,95.401758
3,41.961454,Median of Two Sorted Arrays,median-of-two-sorted-arrays,"[array, binary-search, divide-and-conquer]",hard,4,4,<p>Given two sorted arrays <code>nums1</code> ...,https://leetcode.com/problems/median-of-two-so...,28998,3261,89.891193
4,34.741408,Longest Palindromic Substring,longest-palindromic-substring,"[two-pointers, string, dynamic-programming]",medium,5,5,"<p>Given a string <code>s</code>, return <em>t...",https://leetcode.com/problems/longest-palindro...,29884,1837,94.208884


In [2]:
difficulty_map = {"easy" : 1, "medium" : 2, "hard" : 3}

df["difficulty"] = df["difficulty"].map(difficulty_map)
df.head()

,accuracy,title,titleSlug,topics,difficulty,questionId,questionFrontendId,question,link,likes,dislikes,likability
0,54.199137,Two Sum,two-sum,"[array, hash-table]",1,1,1,<p>Given an array of integers <code>nums</code...,https://leetcode.com/problems/two-sum,58795,2097,96.556198
1,44.566495,Add Two Numbers,add-two-numbers,"[linked-list, math, recursion]",2,2,2,<p>You are given two <strong>non-empty</strong...,https://leetcode.com/problems/add-two-numbers,32082,6445,83.271472
2,35.730692,Longest Substring Without Repeating Characters,longest-substring-without-repeating-characters,"[hash-table, string, sliding-window]",2,3,3,"<p>Given a string <code>s</code>, find the len...",https://leetcode.com/problems/longest-substrin...,40582,1956,95.401758
3,41.961454,Median of Two Sorted Arrays,median-of-two-sorted-arrays,"[array, binary-search, divide-and-conquer]",3,4,4,<p>Given two sorted arrays <code>nums1</code> ...,https://leetcode.com/problems/median-of-two-so...,28998,3261,89.891193
4,34.741408,Longest Palindromic Substring,longest-palindromic-substring,"[two-pointers, string, dynamic-programming]",2,5,5,"<p>Given a string <code>s</code>, return <em>t...",https://leetcode.com/problems/longest-palindro...,29884,1837,94.208884


In [3]:
df.isna().sum()

accuracy                0
title                   0
titleSlug               0
topics                  0
difficulty              0
questionId              0
questionFrontendId      0
question              685
link                    0
likes                   0
dislikes                0
likability              0
dtype: int64

In [ ]:
df.dropna(inplace=True) # Remove Premium Problems
df.count()

accuracy              2668
title                 2668
titleSlug             2668
topics                2668
difficulty            2668
questionId            2668
questionFrontendId    2668
question              2668
link                  2668
likes                 2668
dislikes              2668
likability            2668
dtype: int64

In [5]:
# Reset index to avoid KeyErrors
df.reset_index(drop=True, inplace=True)
df.to_json("updated_data.json")

## **Model**

In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import networkx as nx

class QuestionRecommender:
    """
    A system for recommending questions based on their content similarity, topic modeling, and 
    user engagement metrics (likability and accuracy). It uses a Markov Random Field (MRF) to 
    model relationships between questions and employs belief propagation for inference.
    """
    def __init__(self, file_path="updated_data.json"):
        """
        Initialize the recommender system by loading data, preprocessing it, and
        calculating matrices for similarity, topic modeling, and potentials. 
        Also constructs a Markov Random Field and a recommendation graph.
        
        Args:
            file_path (str): Path to the JSON file containing question data.
        """
        self.df = self.load_and_preprocess_data(file_path)
        self.similarity_matrix = None
        self.topic_matrix = None
        self.potential_matrix = None
        self.mrf = None
        self.G = None

        self.calculate_similarity_matrix()
        self.calculate_topic_matrix()
        self.calculate_potential_matrix()
        self.build_mrf()
        self.build_graph()

    def load_and_preprocess_data(self, file_path):
        """
        Load the dataset from a JSON file and preprocess it for further analysis.

        Args:
            file_path (str): Path to the dataset file.

        Returns:
            pd.DataFrame: Preprocessed DataFrame containing question data.
        """
        df = pd.read_json(file_path)
        df = self.preprocess_data(df)
        return df

    def preprocess_data(self, df):
        """
        Normalize the 'likability' and 'accuracy' columns using Min-Max Scaling.

        Args:
            df (pd.DataFrame): The input DataFrame.

        Returns:
            pd.DataFrame: The DataFrame with normalized 'likability' and 'accuracy'.
        """
        scaler = MinMaxScaler()
        df[['likability', 'accuracy']] = scaler.fit_transform(df[['likability', 'accuracy']])
        return df

    def calculate_similarity_matrix(self):
        """
        Calculate the pairwise similarity between questions using TF-IDF vectorization
        and cosine similarity. Results are stored in `self.similarity_matrix`.
        """
        question_vectors = TfidfVectorizer().fit_transform(self.df['question'])
        self.similarity_matrix = cosine_similarity(question_vectors)

    def calculate_topic_matrix(self):
        """
        Perform custom topic modeling on the questions to identify latent topics.
        Results are stored in `self.topic_matrix`.
        """
        self.topic_matrix = self.custom_topic_model(self.df['question'].tolist(), n_topics=3)

    def calculate_potential_matrix(self):
        """
        Compute the potential matrix for the Markov Random Field based on joint probabilities 
        of accuracy bins and difficulty levels. Uses binning to discretize accuracy.
        """
        bins = np.linspace(0, 1, 5)  # Divide accuracy into 4 bins
        self.df['accuracy_bin'] = np.digitize(self.df['accuracy'], bins) - 1
        joint_prob = pd.crosstab(self.df['accuracy_bin'], self.df['difficulty'], normalize='all')
        self.potential_matrix = np.zeros((4, 3))  # 4 bins for accuracy, 3 difficulty levels

        for acc_bin in range(4):  # Accuracy bins
            for diff in range(1, 4):  # Difficulty levels
                if diff in joint_prob.columns and acc_bin in joint_prob.index:
                    self.potential_matrix[acc_bin, diff - 1] = joint_prob.loc[acc_bin, diff]

    class MarkovRandomField:
        """
        A class representing a Markov Random Field (MRF), which consists of nodes (questions) 
        and edges (relationships between questions) with associated potentials.
        """
        def __init__(self):
            """
            Initialize the MRF with empty node and edge dictionaries and a dictionary
            for edge potentials.
            """
            self.edges = {}
            self.nodes = {}
            self.edge_potentials = {}

        def add_edge(self, node1, node2, potential):
            """
            Add an edge between two nodes with a specified potential matrix.

            Args:
                node1 (str): The first node.
                node2 (str): The second node.
                potential (np.ndarray): The edge potential matrix.
            """
            if node1 not in self.nodes:
                self.nodes[node1] = {}
            if node2 not in self.nodes:
                self.nodes[node2] = {}
            self.edges[(node1, node2)] = True
            self.edge_potentials[(node1, node2)] = potential

        def compute_potential(self, node1, node2, acc_bin, diff):
            """
            Compute the potential for a given pair of nodes based on accuracy bin 
            and difficulty level.

            Args:
                node1 (str): The first node.
                node2 (str): The second node.
                acc_bin (int): The accuracy bin index.
                diff (int): The difficulty index.

            Returns:
                float: The potential value for the specified parameters.
            """
            if (node1, node2) in self.edge_potentials:
                return self.edge_potentials[(node1, node2)][acc_bin, diff]
            return 1.0

    def belief_propagation(self, max_iter=10):
        """
        Run belief propagation on the MRF to compute refined potentials.

        Args:
            max_iter (int): Maximum number of iterations for belief propagation.

        Returns:
            dict: A dictionary containing messages for each edge in the MRF.
        """
        messages = {}
        for (node1, node2) in self.mrf.edges:
            messages[(node1, node2)] = np.ones((4, 3))  # 4 bins, 3 difficulty levels

        for _ in range(max_iter):
            for (node1, node2) in self.mrf.edges:
                incoming_messages = np.ones((4, 3))
                for nbr in self.mrf.nodes[node1]:
                    if nbr != node2:
                        incoming_messages *= messages[(nbr, node1)]
                
                new_message = np.zeros((4, 3))
                for acc_bin in range(4):
                    for diff in range(3):
                        new_message[acc_bin, diff] = (
                            self.mrf.compute_potential(node1, node2, acc_bin, diff)
                            * incoming_messages[acc_bin, diff]
                        )
                messages[(node1, node2)] = new_message / new_message.sum()

        return messages

    def build_mrf(self):
        """
        Build the Markov Random Field by adding nodes (questions) and edges with
        potentials based on the accuracy bins and difficulty levels.
        """
        self.mrf = self.MarkovRandomField()
        
        for idx, row in self.df.iterrows():
            self.mrf.nodes[row['titleSlug']] = {
                'accuracy_bin': row['accuracy_bin'],
                'difficulty': row['difficulty']
            }

        for i in range(len(self.df)):
            for j in range(i + 1, len(self.df)):
                acc_bin_i = self.df.loc[i, 'accuracy_bin']
                diff_i = self.df.loc[i, 'difficulty']
                acc_bin_j = self.df.loc[j, 'accuracy_bin']
                diff_j = self.df.loc[j, 'difficulty']
                
                edge_potential = np.zeros((4, 3))
                for acc_bin in range(4):
                    for diff in range(3):
                        if acc_bin == acc_bin_i or acc_bin == acc_bin_j:
                            edge_potential[acc_bin, diff] += 0.5
                        if diff == diff_i or diff == diff_j:
                            edge_potential[acc_bin, diff] += 0.5
                
                edge_potential /= edge_potential.sum()
                self.mrf.add_edge(self.df.loc[i, 'titleSlug'], self.df.loc[j, 'titleSlug'], edge_potential)

    def custom_topic_model(self, corpus, n_topics=2, alpha=0.1, beta=0.01):
        """
        Perform a simple topic modeling process using Gibbs sampling.

        Args:
            corpus (list of str): List of documents (questions).
            n_topics (int): Number of latent topics.
            alpha (float): Dirichlet prior for topics.
            beta (float): Dirichlet prior for words.

        Returns:
            np.ndarray: Document-topic matrix.
        """
        vocab = {word: idx for idx, word in enumerate(set(' '.join(corpus).split()))}
        word_topic_matrix = np.zeros((len(vocab), n_topics))
        doc_topic_matrix = np.zeros((len(corpus), n_topics))

        doc_word_topic = []
        for i, doc in enumerate(corpus):
            topics = []
            for word in doc.split():
                topic = np.random.randint(0, n_topics)
                topics.append(topic)
                word_topic_matrix[vocab[word], topic] += 1
                doc_topic_matrix[i, topic] += 1
            doc_word_topic.append(topics)

        for _ in range(50):
            for d, doc in enumerate(corpus):
                for i, word in enumerate(doc.split()):
                    topic = doc_word_topic[d][i]
                    word_topic_matrix[vocab[word], topic] -= 1
                    doc_topic_matrix[d, topic] -= 1

                    topic_dist = (word_topic_matrix[vocab[word], :] + beta) * (doc_topic_matrix[d, :] + alpha)
                    new_topic = np.argmax(topic_dist / topic_dist.sum())

                    doc_word_topic[d][i] = new_topic
                    word_topic_matrix[vocab[word], new_topic] += 1
                    doc_topic_matrix[d, new_topic] += 1

        return doc_topic_matrix / doc_topic_matrix.sum(axis=1, keepdims=True)

    def topic_overlap(self, doc1_topics, doc2_topics):
        """
        Calculate the topic overlap score between two documents.

        Args:
            doc1_topics (np.ndarray): Topic distribution for the first document.
            doc2_topics (np.ndarray): Topic distribution for the second document.

        Returns:
            float: Sum of the minimum topic probabilities for all topics.
        """
        return np.sum(np.minimum(doc1_topics, doc2_topics))

    def build_graph(self):
        """
        Construct a graph where nodes represent questions, and edges are weighted by
        a combination of content similarity and topic overlap scores.
        """
        self.G = nx.Graph()
        for idx, row in self.df.iterrows():
            self.G.add_node(row['titleSlug'], difficulty=row['difficulty'], likability=row['likability'], accuracy=row['accuracy'])

        for i in range(len(self.df)):
            for j in range(i + 1, len(self.df)):
                content_similarity = self.similarity_matrix[i, j]
                topic_overlap_score = self.topic_overlap(self.topic_matrix[i], self.topic_matrix[j])
                combined_weight = (content_similarity + topic_overlap_score) / 2
                self.G.add_edge(self.df.loc[i, 'titleSlug'], self.df.loc[j, 'titleSlug'], weight=combined_weight)

    def recommend_questions(self, solved_questions, top_n=3):
        """
        Recommend questions based on solved questions using the constructed graph.

        Args:
            solved_questions (list of str): List of solved question IDs.
            top_n (int): Number of recommendations to return.

        Returns:
            list of tuple: Recommended questions with their weights, sorted in descending order.
        """
        recommendations = {}
        for solved in solved_questions:
            if solved not in self.G:
                continue
            neighbors = sorted(self.G[solved].items(), key=lambda x: x[1]['weight'], reverse=True)
            for neighbor, _ in neighbors:
                if neighbor not in solved_questions:
                    recommendations[neighbor] = self.G[solved][neighbor]['weight']

        return sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:top_n]

# Example usage:
recommender = QuestionRecommender()  # Load and preprocess once
solved_questions = ["two-sum", "add-two-numbers"]
top_n = 10
recommended = recommender.recommend_questions(solved_questions, top_n)
print("Recommendations:", recommended)


Recommendations: [('add-two-numbers-ii', 0.9320920323613826), ('convert-binary-number-in-a-linked-list-to-integer', 0.7187524165059036), ('swapping-nodes-in-a-linked-list', 0.7095636998032739), ('merge-nodes-in-between-zeros', 0.7025619353829341), ('fibonacci-number', 0.6983446250833296), ('double-a-number-represented-as-a-linked-list', 0.6978497677350213), ('consecutive-numbers-sum', 0.6960625240386483), ('find-n-unique-integers-sum-up-to-zero', 0.6926349573643795), ('rotate-list', 0.6918581924456334), ('sort-list', 0.6905761656570419)]


In [ ]:
# Example usage:
solved_questions = ["integer-to-roman", "same-tree"]
top_n = 10
recommended = recommender.recommend_questions(solved_questions, top_n)
print("Recommendations:", recommended)

Recommendations: [('find-bottom-left-tree-value', 0.780216719981049), ('find-duplicate-subtrees', 0.7622902321002853), ('balanced-binary-tree', 0.7617745734394711), ('validate-binary-search-tree', 0.743986582792115), ('unique-binary-search-trees', 0.736006191091326), ('merge-bsts-to-create-single-bst', 0.7304450597795183), ('find-largest-value-in-each-tree-row', 0.7255177163400416), ('rotate-list', 0.72511478849944), ('two-sum-iv-input-is-a-bst', 0.7245592498135529), ('kth-smallest-element-in-a-bst', 0.7243658625422833)]
